In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

In [2]:
secrets_file = open("C:\\Users\\Acer\Desktop\\Ironhack Berlin\\ZAJECIA\\week6\\secrets.txt","r") 

string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
        
# initializing SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [3]:
# dataframes with playlists
top100 = pd.read_csv('top100.csv')
playlist_clustered = pd.read_csv('playlist_with_clusters.csv')

In [4]:
# loading pre-trained models
scaler = pickle.load(open('scaler.pkl','rb'))
kmeans = pickle.load(open('kmean.pkl', 'rb'))

In [5]:
def get_track_features(song_name, artist_name):
    try:
        # searching for the track on Spotify
        results = sp.search(q=f'{song_name} artist:{artist_name}', type='track', limit=1)

        # if there are results
        if results['tracks']['items']:
            # getting ID of the track (first found)
            track_id = results['tracks']['items'][0]['id']

            # fetching audio features
            audio_features = sp.audio_features(track_id)[0]

            # selecting the attributes
            selected_attributes = ['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                                    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                    'duration_ms', 'time_signature']

            # creating dictionary with selected attributes
            features_dict = {attr: audio_features[attr] for attr in selected_attributes}

            # adding additional information
            features_dict['song_id'] = audio_features['id']
            features_dict['song_name'] = sp.track(track_id)['name']
            features_dict['artist'] = sp.track(track_id)['artists'][0]['name']

            # creating dataframe
            df = pd.DataFrame([features_dict])

            # reordering columns
            df = df[['song_id', 'song_name', 'artist', 'danceability', 'energy', 'key', 'loudness',
                     'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                     'valence', 'tempo', 'duration_ms', 'time_signature']]

            # dropping unnecessary columns
            new_df = df.drop(['song_id', 'song_name', 'artist', 'time_signature'], axis=1)

            # standardizing new song
            std_new_song = scaler.transform(new_df)

            # predicting cluster for the new song
            new_cluster = kmeans.predict(std_new_song)

            # filtering dataframe to songs in the same cluster
            df_cluster = playlist_clustered[playlist_clustered['cluster'] == list(new_cluster)[0]]

            # randomly selecting a song from the cluster (sample(n=1) returns random row from the dataframe)
            random_recommendation = df_cluster.sample(n=1)

            # recommendation
            print(f'Your recommendation: \nSong: {random_recommendation["song_name"].values[0]}\nArtist: {random_recommendation["artist"].values[0]}')

        else:
            # if there are no results - exit
            print('We have no other recommendations similar to your song.')

    except Exception as e:
        print(f'An error occurred: {str(e)}')

In [6]:
while True:
    try:
        # getting user input
        input_song = input("Enter song title: ").lower()
        input_artist = input("Enter artist: ").lower()

        # converting song titles in top100 to lowercase
        top100_lower = top100['song_title'].str.lower()

        if input_song in top100_lower.values:
            # if the input song is in top100, display a random recommendation from top100
            top100_filtered = top100.loc[top100_lower != input_song]
            random_recommendation = top100_filtered.sample(n=1)
            print(f'Your recommendation: \nSong: {random_recommendation["song_title"].values[0]}\nArtist: {random_recommendation["artist"].values[0]}')
        else:
            # if the input song is not in top100, get recommendations using the main function
            get_track_features(input_song, input_artist)

        # asking if the user wants another recommendation
        next_input = input('\nAnother song recommendation? (type "Y" to continue, any other key to exit): ')
        print()
        if next_input.lower() != "y":
            break

    except Exception as e:
        print(f'An error occurred: {str(e)}')

Enter song title: houdini
Enter artist: dua lipa
Your recommendation: 
Song: You Broke My Heart
Artist: Drake

Another song recommendation? (type "Y" to continue, any other key to exit): y

Enter song title: lovin on me
Enter artist: jack harlow
Your recommendation: 
Song: Need A Favor
Artist: Jelly Roll

Another song recommendation? (type "Y" to continue, any other key to exit): n

